<a href="https://colab.research.google.com/github/wuphwu/SCC_Auto/blob/main/ServerAccount_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

用於彙整伺服器產生的Account List文字檔
要手動上傳(多個)Account List文字檔到"/content/data"


In [7]:
import os
import shutil
import csv
import pandas as pd
from google.colab import files

In [8]:
directory = "/content/data"

# 如果目錄存在，則刪除
if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"目錄 {directory} 已刪除")

# 重新建立目錄
os.makedirs(directory)
print(f"目錄 {directory} 已建立")

目錄 /content/data 已刪除
目錄 /content/data 已建立


In [9]:
# 讓使用者上傳檔案
uploaded = files.upload()

# 移動檔案到 `/content/data/`
for filename in uploaded.keys():
    os.rename(filename, os.path.join(directory, filename))

Saving SCC-REC-NAS2.txt to SCC-REC-NAS2.txt
Saving SCC-HYPERV-FSSL.txt to SCC-HYPERV-FSSL.txt
Saving SCC-PPCM-DB2-BK.txt to SCC-PPCM-DB2-BK.txt
Saving SCC-PRRE-APe.txt to SCC-PRRE-APe.txt
Saving SCC-DL380-0049.txt to SCC-DL380-0049.txt
Saving SCC-DL380-0063.txt to SCC-DL380-0063.txt
Saving SCC-DDOC-AP1.txt to SCC-DDOC-AP1.txt
Saving SCC-HYPERV-04.txt to SCC-HYPERV-04.txt
Saving SCC-TAIPEI-BAK.txt to SCC-TAIPEI-BAK.txt
Saving SCC-BAKSVR-01.txt to SCC-BAKSVR-01.txt
Saving SCC-REC-0057.txt to SCC-REC-0057.txt
Saving SCC-HYPERV-06e.txt to SCC-HYPERV-06e.txt
Saving SCC-FSSL-AP.txt to SCC-FSSL-AP.txt
Saving SCC-FSBL-AP.txt to SCC-FSBL-AP.txt
Saving SCC-REC-NAS1.txt to SCC-REC-NAS1.txt
Saving SCC-PTIC-0050.txt to SCC-PTIC-0050.txt
Saving SCC-CDS-AP.txt to SCC-CDS-AP.txt
Saving SCC-TBKN-APDB.txt to SCC-TBKN-APDB.txt
Saving SCC-DL380-0048-e.txt to SCC-DL380-0048-e.txt
Saving SCC-DDOC-AP2-E.txt to SCC-DDOC-AP2-E.txt
Saving SCC-PSWS-AP.txt to SCC-PSWS-AP.txt
Saving SCC-NATF-APe.txt to SCC-NATF-AP

In [10]:
def merge_txt_to_csv(txt_folder, output_csv):
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['server name', 'account name', 'account type', 'group'])

        for file in os.listdir(txt_folder):
            if file.endswith('.txt'):
                server_name = os.path.splitext(file)[0]  # 檔名去掉 .txt
                file_path = os.path.join(txt_folder, file)

                with open(file_path, 'r', encoding='utf-16' ) as txt_file:
                  datatype = 0 # 0:一開始的資料  1:本機帳號資料 2:DB帳號資料
                  for line in txt_file:
                    parts = line.strip().split(' ', 2)  # 按空格分割，最多分成 3 部分
                    if datatype == 0 and parts[0] == '----':
                      datatype = 1
                    elif datatype == 1 and len(parts) == 3 and parts[0] != '無SQL': # name True Administrators, Users
                      result = parts[2].replace("True", "").strip()
                      if "Administrators" in result:
                        result = "管理者帳號"
                      writer.writerow([server_name, parts[0], "本機" , result])
                    elif datatype == 1 and parts[0] == '資料庫使用者:':
                      datatype = 2
                    elif datatype == 2 and parts[2] == '- WINDOWS_LOGIN':
                      dbaccount = parts[1].strip().split('\\', 1)  # 按\分割，最多分成 ２ 部分
                      writer.writerow([server_name, dbaccount[1], 'SQL', "管理者帳號"])
                    elif datatype == 2 and parts[2] == '- SQL_LOGIN':
                      writer.writerow([server_name, parts[1], 'SQL', "管理者帳號"])
def csv_format(input_csv, output_csv):
  # 讀取 CSV，確保 , 分隔符
  # 資料庫帳號資料省略目標DB，故會有重複多筆，先進行deduplicate
  df = pd.read_csv(input_csv, sep=",", dtype=str).drop_duplicates()

  # 確保 `account type` 欄位是字串，並填補 NaN
  df["account type"] = df["account type"].astype(str).fillna("")
  df["group"] = df["group"].astype(str).fillna("")

  # 合併相同的 `server name` 和 `account name`
  df_merged = df.groupby(["server name", "account name", "group"], as_index=False)["account type"].agg(lambda x: ", ".join(filter(None, x)))

  # 重新排列欄位
  new_order = ["server name", "account name", "account type","group"]
  df_merged = df_merged[new_order]  # 按照新順序重新排序

  # 儲存新的 CSV
  df_merged.to_csv(output_csv, sep=",", index=False, encoding="utf-8-sig")






In [11]:
# 設定資料夾與輸出檔案路徑
txt_folder = directory  # 本地工作目錄中的資料夾
output_csv = '/content/merged_accounts.csv'  # 存儲 CSV 檔案的路徑

merge_txt_to_csv(txt_folder, output_csv)
# print(f'合併完成，CSV 檔案儲存於: {output_csv}')


final_csv = "/content/accounts_final.csv" # 輸出的 CSV 檔案
csv_format(output_csv, final_csv)
print(f"合併完成，CSV 檔案儲存於: {final_csv}")

合併完成，CSV 檔案儲存於: /content/accounts_final.csv


In [12]:
# 刪除 temp_file
try:
    temp_file = "/content/merged_accounts.csv"
    if os.path.exists(temp_file):
        os.remove(temp_file)
        print(f"{temp_file} 已成功刪除")
    else:
        print(f"{temp_file} 不存在，無需刪除")
except Exception as e:
    print(f"刪除 {temp_file} 時發生錯誤: {e}")

/content/merged_accounts.csv 已成功刪除
